In [98]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [112]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.set_index('PassengerId', inplace=True)
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [113]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.set_index('PassengerId', inplace=True)
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [114]:
import re

def missing_age(df):
    df['Title'] = [re.search(r'(.*\,\s)(.*)(\..*)', name).group(2) for name in df.Name]
    mask = (df.Age.isna())
    df.loc[mask & (df.Title=='Master'), 'Age'] = 10
    df.loc[mask & (df.Title=='Dr'), 'Age'] = 50
    df.loc[mask & (df.Title=='Mr'), 'Age'] = 50
    df.loc[mask & (df.Title=='Mrs'), 'Age'] = 50
    df.loc[mask & (df.Title=='Miss'), 'Age'] = 10
    df.loc[mask & (df.Title=='Ms'), 'Age'] = 50
    return

missing_age(train_data)
missing_age(test_data)

In [115]:
train_data[train_data.Age.isna() == True]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,


In [116]:
test_data[test_data.Age.isna() == True]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,


In [117]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]
X_test, X_train, y_test, y_train = train_test_split(train_data, y)

In [118]:
features = ["Pclass", "Sex", "SibSp", "Parch", "Age"]
X_train = pd.get_dummies(X_train[features])
X_test = pd.get_dummies(X_test[features])
test_data = pd.get_dummies(test_data[features])


In [120]:
model = RandomForestClassifier(n_estimators=100, max_depth=5)
model.fit(X_train, y_train)
predict = model.predict(X_test)
predict_train = model.predict(X_train)


#output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
#output.to_csv('submission.csv', index=False)
#print("Your submission was successfully saved!")

In [121]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(predict,y_test))
print(classification_report(predict,y_test))
print('Train set accuracy: ' + str(accuracy_score(predict_train, y_train)))
print('Test set accuracy: ' + str(accuracy_score(predict, y_test)))


[[372  91]
 [ 37 168]]
              precision    recall  f1-score   support

           0       0.91      0.80      0.85       463
           1       0.65      0.82      0.72       205

    accuracy                           0.81       668
   macro avg       0.78      0.81      0.79       668
weighted avg       0.83      0.81      0.81       668

Train set accuracy: 0.874439461883408
Test set accuracy: 0.8083832335329342


In [62]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [10, 100, 500]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [2, 5, 10, 50, 80, None]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [63]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
model_search = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = model_search,
                               param_distributions = random_grid, n_iter = 100, cv = 3,
                               verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits

[CV] END bootstrap=False, max_depth=50, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=False, max_depth=50, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=False, max_depth=5, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=False, max_depth=5, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=False, max_depth=5, max_features=log2, min_samples_leaf=10, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=2, max_features=sqrt, min_samples_leaf=10, min_samples_split=5, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=2, max_features=sqrt, min_samples_leaf=10, min_samples_split=5, n_estimator

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [2, 5, 10, 50, 80, None],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': [2, 4, 10],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 100, 500]},
                   random_state=42, verbose=2)

In [122]:
rf_random.best_params_

{'n_estimators': 100,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'log2',
 'max_depth': 10,
 'bootstrap': False}

In [123]:
params = {'n_estimators': 10,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

base_model = RandomForestClassifier(**params)
base_model.fit(X_train, y_train)
predict_base = base_model.predict(X_test)
base_accuracy = accuracy_score(predict_base, y_test)
base_accuracy

0.8068862275449101

In [124]:
best_random = rf_random.best_estimator_
best_random.fit(X_train, y_train)
predict_random = best_random.predict(X_test)
random_accuracy = accuracy_score(predict_random, y_test)
random_accuracy

0.7904191616766467

In [129]:
predict = base_model.predict(test_data)
output = pd.DataFrame({'PassengerId': test_data.index, 'Survived': predict})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [126]:
test_data

,Pclass,SibSp,Parch,Age,Sex_female,Sex_male
PassengerId,,,,,,
892,3,0,0,34.5,0,1
893,3,1,0,47.0,1,0
894,2,0,0,62.0,0,1
895,3,0,0,27.0,0,1
896,3,1,1,22.0,1,0
...,...,...,...,...,...,...
1305,3,0,0,50.0,0,1
1306,1,0,0,39.0,1,0
1307,3,0,0,38.5,0,1


In [92]:
test_data

,Pclass,SibSp,Parch,Age,Sex_female,Sex_male
0,3,0,0,34.5,0,1
1,3,1,0,47.0,1,0
2,2,0,0,62.0,0,1
3,3,0,0,27.0,0,1
4,3,1,1,22.0,1,0
...,...,...,...,...,...,...
413,3,0,0,50.0,0,1
414,1,0,0,39.0,1,0
415,3,0,0,38.5,0,1
416,3,0,0,50.0,0,1



[CV] END bootstrap=True, max_depth=50, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=5, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=   1.5s
[CV] END bootstrap=False, max_depth=50, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END bootstrap=True, max_depth=80, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=   1.4s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=   1.0s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=500; total time=   1.3s
[CV] END bootstrap=True, max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=500; total time=   1.3s
[CV] END bootstrap=False, max_depth=None, max_fe